In [ ]:
import json
import re
from pathlib import Path

import pandas as pd

def parse_properties_cell(s: str) -> dict:
    if pd.isna(s):
        return {}
    s = str(s).strip()
    # If the whole cell is quoted, strip outer quotes
    if (s.startswith('"') and s.endswith('"')) or (s.startswith("'") and s.endswith("'")):
        s = s[1:-1].strip()
    # Some exports double-quote JSON quotes -> replace doubled quotes with single quotes
    s = s.replace('""', '"')
    # Sometimes there are stray leading/trailing characters before/after the JSON object;
    # try to extract the {...} substring
    m = re.search(r'\{.*\}', s, flags=re.DOTALL)
    if m:
        s = m.group(0)
    # Unescape common escape sequences
    s = s.replace(r'\"', '"').replace(r"\\", "\\")
    # Try JSON parse, fallback to permissive fixes
    try:
        return json.loads(s)
    except Exception:
        # Replace single quotes for JSON-like strings
        s2 = re.sub(r"(?<!:)\s*'([^']*)'\s*(?=[:,}])", r'"\1"', s)
        s2 = s2.replace("'", '"')
        try:
            return json.loads(s2)
        except Exception:
            # last resort: evaluate simple dict-like string (unsafe for untrusted input)
            try:
                from ast import literal_eval
                return dict(literal_eval(s))
            except Exception:
                return {}

def expand_properties(df: pd.DataFrame, properties_col: str = "properties") -> pd.DataFrame:
    rows = []
    for _, r in df.iterrows():
        base = r.to_dict()
        props = parse_properties_cell(base.get(properties_col, ""))
        merged = {**base, **props}
        rows.append(merged)
    out = pd.DataFrame(rows)
    if properties_col in out.columns:
        out = out.drop(columns=[properties_col])
    # Convert numeric-like fields where appropriate
    for col in ("semester_number", "number"):
        if col in out.columns:
            out[col] = pd.to_numeric(out[col], errors="coerce").astype("Int64")
    return out

def main():
    src = Path(r"C:\Power-bi-projects\UniversityData\student.xlsx")
    if not src.exists():
        raise SystemExit(f"Source not found: {src}")
    df = pd.read_excel(src)
    cleaned = expand_properties(df, properties_col="properties")
    desired = ["id", "name", "gender", "date_of_birth", "enrollment_semester", "semester_number", "major", "number"]
    cols_existing = [c for c in desired if c in cleaned.columns]
    result = cleaned[cols_existing]
    out = src.with_name(src.stem + "_cleaned.xlsx")
    result.to_excel(out, index=False)
    print("Wrote:", out)

if __name__ == "__main__":
    main()

Wrote: C:\Power-bi-projects\UniversityData\student_cleaned.xlsx
